# Adversarial Simulator for Question Answering

## Objective

This tutorial provides a step-by-step guide on how to leverage Generative AI's simulator to simulate an adversarial question answering scenario against an Azure OpenAI chat completions endpoint.

This tutorial uses the following Azure AI services:

- Generative AI's simulator


## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates a simulated adversarial question answering and evaluation. It is important to have access to AzureOpenAI credentials and an AzureAI project.


## Before you begin



### Installation

Install the following packages required to execute this notebook. 



In [ ]:
# Install the packages
%pip install azure-ai-generative[simulator,evaluate]

### Parameters and imports

In [ ]:
from pathlib import Path
from azure.ai.generative.evaluate import evaluate
import json
from azure.ai.generative.synthetic.simulator import Simulator
from azure.ai.resources.client import AIClient
from azure.identity import DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from openai import AsyncAzureOpenAI

sub = ""
rg = ""
project_name = ""
oai_client = AsyncAzureOpenAI(api_key="", azure_endpoint="", api_version="2023-12-01-preview")

## Simulate the data

In [ ]:
async_oai_chat_completion_fn = oai_client.chat.completions.create
client = AIClient(
    subscription_id=sub, resource_group_name=rg, project_name=project_name, credential=DefaultAzureCredential()
)

simulator = Simulator.from_fn(fn=async_oai_chat_completion_fn, ai_client=client, model="gpt-4", max_tokens=300)
adv_template = Simulator.get_template("adv_qa")

## Run the simulator

In [ ]:
outputs = await simulator.simulate_async(
    adv_template, max_conversation_turns=1, max_simulation_results=3, api_call_delay_sec=10
)

## Print the outputs

In [ ]:
for line in outputs:
    print(json.dumps(line, indent=2))

## Store outputs in file for eval

In [ ]:
eval_name = "conv-oai-qa-harm-eval"
file_name = f"\{eval_name}.jsonl"
jsonl_object = outputs.to_eval_qa_json_lines()
with Path.open(file_name, "w") as f:
    f.write(jsonl_object)

## Run evaluate

In [ ]:
tracking_uri = client.tracking_uri
connection = client.connections.get("Default_AzureOpenAI")
config = AzureOpenAIModelConfiguration.from_connection(connection, model_name="gpt-4", deployment_name="gpt-4")
data_path = Path.cwd() / f"\{eval_name}.jsonl"
output_path = Path.cwd() / "downloaded_artifacts" / "remote"
result = evaluate(
    evaluation_name=eval_name,
    data=data_path,
    task_type="qa",
    metrics_list=[
        "hate_unfairness",
        "violence",
        "self_harm",
        "sexual",
        "gpt_groundedness",
        "gpt_relevance",
        "gpt_coherence",
        "gpt_fluency",
        "gpt_similarity",
    ],
    model_config=config,
    data_mapping={
        "questions": "question",
        "answer": "answer",
    },
    tracking_uri=tracking_uri,
    output_path=output_path,
)
print(result)
print(result.metrics_summary)  # will print the defect rate for each content harm
print("Studio URL")
print(result.studio_url)